In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2

import pickle 
import tensorflow as tf 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D

In [2]:
Datadir = 'C:/Users/shree/Downloads/PetImages'
Categories = ['Dog','Cat']

img_size = 70
training_data = []

def create_training_data():
    for Category in Categories:
        path = os.path.join(Datadir,Category)  # path to cats and dogs dir
        # print(path)
        class_num = Categories.index(Category) # Assigning 0-dog and 1-cat for classification purpose

        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_GRAYSCALE)
                new_array = cv2.resize(img_array, (img_size,img_size))
                training_data.append([new_array,class_num])
            except Exception as e:  # Images that are broken will be thrown as expection and will be discarded
                pass

create_training_data()
print(len(training_data))

24946


In [3]:
## Important points
# 1) It is important to have balanced data i.e. 50% dog and 50% cat to train the model if it is 75% dog and 25% cat,
# then it will be difficult for the model to predict duing testing 
# 2) We need to shuffle the data while training. If we train all the dogs together then it will be difficult for the model to 
# wile training all the cats 

import random
random.shuffle(training_data)

# Now that the data is shuffled lets pack it in x and y before we feed it to neural network 
X = []
y = []

for features,label in training_data:
    X.append(features)
    y.append(label)

# Now we have features in list and we can't feed list to the NN, it has to be numpy array
X = np.array(X).reshape(-1, img_size, img_size, 1) 
# -1 is for how many features we have, then shape the array by img-size x img_size and 1 is for gray scale. 
# If we are doing cov NN for colored images then it has to be 3 (RGB) instead of 1
# Pickle in Python is primarily used in serializing and deserializing a Python object structure. 
# In other words, it's the process of converting a Python object into a byte stream to store it in a file/database, maintain program state across sessions, or transport data over the network.

pickle_out = open("X.pickle","wb")
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out = open("y.pickle","wb")
pickle.dump(y, pickle_out)
pickle_out.close()

In [4]:
## Applying Convolutional Neural Network
# Refer to:- https://www.youtube.com/watch?v=zfiSAzpy9NM
print('ram')
X = pickle.load(open("X.pickle","rb"))
y = pickle.load(open("y.pickle","rb"))
# Normalising the data
X = X/255.0

# Convert into np array since validation_split supports only numpy array
#X = np.array(X)
y = np.array(y) 

ram


In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 1)

In [6]:
from scikeras.wrappers import KerasClassifier
from keras.utils import to_categorical
# from sklearn.model_selection import GridSearchCV
import pprint
pp = pprint.PrettyPrinter(indent = 4)

from tensorflow import keras
from keras_tuner import RandomSearch

In [13]:
model = Sequential()
model.add(Conv2D(256, (3,3), input_shape = X.shape[1:])) #X.shape[1:] is beacasue we don't want the number of images, we jsut want the shape of images so we ignore the 1st column
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#model.add(Dropout(0.5))

model.add(Conv2D(256, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#model.add(Dropout(0.5))

# Note that convolution is 2D whereas dense layer is 1D so we need to flatten the data
model.add(Flatten())
model.add(Dense(64))
#model.add(Dropout(0.5))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(
  loss = "binary_crossentropy",
  optimizer = "adam",
  metrics = ['accuracy']
  )

model.fit(X_train, y_train, batch_size = 32, epochs = 5, validation_data=(X_test,y_test))


Epoch 1/5
585/585 [==============================] - 755s 1s/step - loss: 0.6573 - accuracy: 0.6165 - val_loss: 0.5969 - val_accuracy: 0.6888
Epoch 2/5
585/585 [==============================] - 1032s 2s/step - loss: 0.5759 - accuracy: 0.7071 - val_loss: 0.5770 - val_accuracy: 0.7058
Epoch 3/5
585/585 [==============================] - 800s 1s/step - loss: 0.5111 - accuracy: 0.7524 - val_loss: 0.5306 - val_accuracy: 0.7377
Epoch 4/5
585/585 [==============================] - 776s 1s/step - loss: 0.4687 - accuracy: 0.7791 - val_loss: 0.5420 - val_accuracy: 0.7250
Epoch 5/5
585/585 [==============================] - 760s 1s/step - loss: 0.4348 - accuracy: 0.7995 - val_loss: 0.4940 - val_accuracy: 0.7677


In [24]:
X.shape

(24946, 70, 70, 1)

In [14]:
def build_model(hp):                 #hp means hyper parameters
    model=Sequential()
    
    #providing the range for hidden layers  
    #providing range for number of neurons in hidden layers
    model.add(Conv2D(filters=hp.Int('filter',min_value=32,max_value=256,step=32),kernel_size=(3,3),activation=hp.Choice("activation", ["relu", "tanh"]), input_shape = X.shape[1:]))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(rate=0.3))
    
    model.add(Conv2D(filters=hp.Int('filter',min_value=32,max_value=256,step=32),kernel_size=(3,3), activation=hp.Choice("activation", ["relu", "tanh"])))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(rate=0.3))
    
    model.add(Conv2D(filters=hp.Int('filter',min_value=32,max_value=256,step=32),kernel_size=(3,3), activation=hp.Choice("activation", ["relu", "tanh"])))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(rate=0.3))

    # Note that convolution is 2D whereas dense layer is 1D so we need to flatten the data
    model.add(Flatten())
    model.add(Dense(units=hp.Int('num_of_neurons',min_value=32,max_value=128,step=32)))
    
    #output layer
    model.add(Dense(1, activation='sigmoid'))    
    
    #compiling the model
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2, 1e-3, 1e-4])),   #tuning learning rate
                  loss='binary_crossentropy',metrics=['accuracy'])
    return model



In [22]:
# importing Sequential from keras
from tensorflow.keras.models import Sequential
#importing Dense and Conv2D layers from keras
from tensorflow.keras.layers import Dense,Conv2D

def build_model(hp):
    # create model object
    model = keras.Sequential([
    #adding first convolutional layer    
    keras.layers.Conv2D(
        #adding filter 
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=32),
        # adding filter size or kernel size
        kernel_size=hp.Choice('conv_1_kernel', values = [3,3]),
        #activation function
        activation=hp.Choice("activation", ["relu", "tanh"]),
        input_shape=X.shape[1:]),
    keras.layers(MaxPooling2D(pool_size=(2,2))), 
        
    # adding second convolutional layer 
    keras.layers.Conv2D(
        #adding filter 
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=32),
        #adding filter size or kernel size
        kernel_size=hp.Choice('conv_2_kernel', values = [3,3]),
        #activation function
        activation=hp.Choice("activation", ["relu", "tanh"])
    ),
    keras.layers(MaxPooling2D(pool_size=(2,2))),
        
    # adding flatten layer    
    keras.layers.Flatten(),
    # adding dense layer    
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=32),
        activation=hp.Choice("activation", ["relu", "tanh"])
    ),
    # output layer    
    keras.layers.Dense(1, activation='sigmoid')
    ])
                   
    #compilation of model
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
              loss='binary_crossentropy',
              metrics=['accuracy'])
    return model

In [18]:
#feeding the model and parameters to Random Search
tuner=RandomSearch(build_model,
                   objective='val_accuracy',
                   max_trials=25,
                   executions_per_trial=1,
                   overwrite=True,
                   directory='tuner-CNN',
                   project_name='Mnist')

In [19]:
#tells us how many hyperparameters we are tuning
#in our case it's 4 = layers, neurons, activation function, learning rate
tuner.search_space_summary()

Search space summary
Default search space size: 4
filter (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 256, 'step': 32, 'sampling': None}
activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh'], 'ordered': False}
num_of_neurons (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [20]:
#fitting the tuner
tuner.search(X_train,y_train,epochs=10,validation_data=(X_test,y_test))

Trial 12 Complete [00h 39m 04s]
val_accuracy: 0.5024851560592651

Best val_accuracy So Far: 0.8465608358383179
Total elapsed time: 12h 14m 20s

Search: Running Trial #13

Value             |Best Value So Far |Hyperparameter
64                |64                |filter
relu              |relu              |activation
32                |96                |num_of_neurons
0.01              |0.001             |learning_rate

Epoch 1/10
585/585 [==============================] - 134s 227ms/step - loss: 0.7520 - accuracy: 0.5009 - val_loss: 0.6933 - val_accuracy: 0.4975
Epoch 2/10
585/585 [==============================] - 140s 240ms/step - loss: 0.7048 - accuracy: 0.5093 - val_loss: 0.6932 - val_accuracy: 0.5025
Epoch 3/10
585/585 [==============================] - 5109s 9s/step - loss: 0.6938 - accuracy: 0.4965 - val_loss: 0.6946 - val_accuracy: 0.4975
Epoch 4/10
517/585 [=========================>....] - ETA: 12s - loss: 0.6938 - accuracy: 0.4962

KeyboardInterrupt: 